In [1]:
import pandas as pd
import numpy as np
# from numba import njit

import matplotlib.pyplot as plt
from matplotlib import cm
from numpy import linalg as LA
from tqdm import tqdm
# from numba import njit

from kernel import *

# from sklearn import svm

from kernel_functions import * # gram_phi, count_kuplet_k, count_kuplet_3
from preprocessing import preprocessing

In [2]:
# load all data as the numpy array type
#X = pd.read_csv('data/Xtr1_mat50.csv', sep=' ', header=None).values
X_raw0 = pd.read_csv('data/Xtr0.csv', sep= ' ', header = None).values.reshape((-1))
X_raw1 = pd.read_csv('data/Xtr1.csv', sep=' ', header=None).values.reshape((-1))
X_raw2 = pd.read_csv('data/Xtr2.csv', sep=' ', header=None).values.reshape((-1))

# transform to an array of string
X_valid0 = pd.read_csv('data/Xte0.csv', sep=' ', header=None).values.reshape((-1))
X_valid1 = pd.read_csv('data/Xte1.csv', sep=' ', header=None).values.reshape((-1))
X_valid2 = pd.read_csv('data/Xte1.csv', sep=' ', header=None).values.reshape((-1))


Y0 = pd.read_csv('data/Ytr0.csv', sep=',', header=0)['Bound'].values
Y1 = pd.read_csv('data/Ytr1.csv', sep=',', header=0)['Bound'].values
Y2 = pd.read_csv('data/Ytr2.csv', sep=',', header=0)['Bound'].values

#print('numerical features shape', X.shape)
#print('numerical features first row', X[0])
print('sequences shape: ', X_raw0.shape)
print('sequence first row: ', X_raw0[0])
print('labels shape', Y0.shape)

sequences shape:  (2000,)
sequence first row:  TCCTCAACTTTTATTGGGCCGCTGTGGCACCAGAATCTACGAATGGCGCCCTCTAGAGTTGTGTAAAGAAGTGGCGTCACCTCATTATAAATAAAAGGTTG
labels shape (2000,)


In [3]:
codon2AA = {}
codon2AA["ATT"]="I";codon2AA["ATC"]="I";codon2AA["ATA"]="I"
codon2AA["CTT"]="L";codon2AA["CTC"]="L";codon2AA["CTA"]="L";codon2AA["CTG"]="L";codon2AA["TTA"]="L";codon2AA["TTG"]="L"
codon2AA["GTT"]="V";codon2AA["GTC"]="V";codon2AA["GTA"]="V";codon2AA["GTG"]="V"
codon2AA["TTT"]="F";codon2AA["TTC"]="F"
codon2AA["ATG"]="M"
codon2AA["TGT"]="C";codon2AA["TGC"]="C"
codon2AA["GCT"]="A";codon2AA["GCC"]="A";codon2AA["GCA"]="A";codon2AA["GCG"]="A"
codon2AA["GGT"]="G";codon2AA["GGC"]="G";codon2AA["GGA"]="G";codon2AA["GGG"]="G"
codon2AA["CCT"]="P";codon2AA["CCC"]="P";codon2AA["CCA"]="P";codon2AA["CCG"]="P"
codon2AA["ACT"]="T";codon2AA["ACC"]="T";codon2AA["ACA"]="T";codon2AA["ACG"]="T"
codon2AA["TCT"]="S";codon2AA["TCC"]="S";codon2AA["TCA"]="S";codon2AA["TCG"]="S";codon2AA["AGT"]="S";codon2AA["AGC"]="S"
codon2AA["TAT"]="Y";codon2AA["TAC"]="Y"
codon2AA["TGG"]="W"
codon2AA["CAA"]="Q";codon2AA["CAG"]="Q"
codon2AA["AAT"]="N";codon2AA["AAC"]="N"
codon2AA["CAT"]="H";codon2AA["CAC"]="H"
codon2AA["GAA"]="E";codon2AA["GAG"]="E"
codon2AA["GAT"]="D";codon2AA["GAC"]="D"
codon2AA["AAA"]="K";codon2AA["AAG"]="K"
codon2AA["CGT"]="R";codon2AA["CGC"]="R";codon2AA["CGA"]="R";codon2AA["CGG"]="R";codon2AA["AGA"]="R";codon2AA["AGG"]="R"
codon2AA["TAA"]="";codon2AA["TAG"]="";codon2AA["TGA"]=""

In [4]:
def transform_sequence(seq):
    i=0
    tab1 = ''
    while(i<len(seq)-2):
        code = codon2AA[seq[i] + seq[i+1] + seq[i+2]]
        # if(code==''):
        #     print("stop") Strategie possible, arrêter la séquence pour les codons stop
        tab1 += code
        i+=3
    i=1
    tab2 = ''
    while(i<len(seq)-2):
        code = codon2AA[seq[i] + seq[i+1] + seq[i+2]]
        tab2 += code
        # if(code==''):
        #     print("stop")
        i+=3
    i=2
    tab3 = ''
    while(i<len(seq)-2):
        code = codon2AA[seq[i] + seq[i+1] + seq[i+2]]
        tab3 += code
        # if(code==''):
        #     print("stop")
        i+=3
    
    return [tab1, tab2, tab3]

In [5]:
from kernel import *
import multiprocessing # import Pool


def solve_svm_kernel(X_train, 
                     X_test, 
                     Y_train, 
                     Y_test, 
                     kernel='k_gram_gaussian', 
                     k=3, k2=6, 
                     k1=4,
                     m = 2, # maximum mismatch for msimatch kernel
                     lamb=0.1, 
                     gamma=0.1, 
                     kktreg=1e-9,
                     codon = False
                    ):
    """
    kernel in ['k_gram', 'k_gram_gaussian', 'k_substring', 'local_alignement', 'k_gram_concat', 'k_gram_concat_several']
    """
    assert kernel in ['k_gram', 'k_gram_gaussian', 'k_substring', 'local_alignement',
                      'k_gram_concat', 'k_gram_concat_several', 'diMismatch', 'mismatch']
    N_train = len(X_train)
    N_test = len(X_test)
    X_train_process = X_train
    X_test_process = X_test
    
    if codon:
        for i in range(N_train):
            # print(X_train_process[i])
            X_train_process[i] = transform_sequence(X_train_process[i])
        for i in tqdm(range(N_test)):
            X_test_process[i] = transform_sequence(X_test_process[i])
        
    if kernel in ['k_gram', 'k_gram_gaussian']:
        if codon:
            X_train_process = np.array([count_kuplet_k(x[0], k=k) + count_kuplet_k(x[1], k=k) \
                                        + count_kuplet_k(x[2], k=k) for x in X_train_process])
            X_test_process = np.array([count_kuplet_k(x[0], k=k) + count_kuplet_k(x[1], k=k) \
                                        + count_kuplet_k(x[2], k=k) for x in X_test_process])
        else:
            if k == 3:
                X_train_process = np.array([count_kuplet_3(x) for x in X_train_process])
                X_test_process = np.array([count_kuplet_3(x) for x in X_test_process])
            else:
                X_train_process = np.array([count_kuplet_k(x,k=k) for x in X_train_process])
                X_test_process = np.array([count_kuplet_k(x,k=k) for x in X_test_process])
        
        # Adding 1 for the sake of the bias
        X_train_process = np.concatenate((X_train_process, np.ones((X_train_process.shape[0], 1))), axis=1)
        X_test_process = np.concatenate((X_test_process, np.ones((X_test_process.shape[0], 1))), axis=1)
    
    if kernel=='k_gram_concat':
        print("building kernel...")
        X_train_process1 = np.array([count_kuplet_3(x) for x in X_train_process])
        X_train_process2 = np.array([count_kuplet_k(x, k=k2) for x in X_train_process])
        X_train_process = np.concatenate((X_train_process1, X_train_process2), axis=1)
        X_test_process1 = np.array([count_kuplet_3(x) for x in X_test_process])
        X_test_process2 = np.array([count_kuplet_k(x, k=k2) for x in X_test_process])
        X_test_process = np.concatenate((X_test_process1, X_test_process2), axis=1)
        
        X_train_process = np.concatenate((X_train_process, np.ones((X_train_process.shape[0], 1))), axis=1)
        X_test_process = np.concatenate((X_test_process, np.ones((X_test_process.shape[0], 1))), axis=1)
    
    if kernel=='k_gram_concat_several':
        print("building kernel...")
        if codon:
            X_train_process = np.array([count_kuplet_k(x[0], k=k1) + count_kuplet_k(x[1], k=k1) \
                                        + count_kuplet_k(x[2], k=k1) for x in X_train_process])
            X_test_process = np.array([count_kuplet_k(x[0], k=k1) + count_kuplet_k(x[1], k=k1) \
                                        + count_kuplet_k(x[2], k=k1) for x in X_test_process])
            for k_tmp in range(k1+1, k2):
                X_train_process2 = np.array([count_kuplet_k(x[0], k=k_tmp) + count_kuplet_k(x[1], k=k_tmp) \
                                        + count_kuplet_k(x[2], k=k_tmp) for x in X_train])
                X_test_process2 = np.array([count_kuplet_k(x[0], k=k_tmp) + count_kuplet_k(x[1], k=k_tmp) \
                                        + count_kuplet_k(x[2], k=k_tmp) for x in X_test])
                X_test_process = np.concatenate((X_test_process, X_test_process2), axis=1)
                X_train_process = np.concatenate((X_train_process, X_train_process2), axis=1)
            
        else :
            X_train_process = np.array([count_kuplet_3(x) for x in X_train_process])
            X_test_process = np.array([count_kuplet_3(x) for x in X_test_process])

            for k_tmp in range(k1, k2):
                X_train_process2 = np.array([count_kuplet_k(x, k=k_tmp) for x in X_train_process])
                X_train_process = np.concatenate((X_train_process, X_train_process2), axis=1)
                X_test_process2 = np.array([count_kuplet_k(x, k=k_tmp) for x in X_test_process])
                X_test_process = np.concatenate((X_test_process, X_test_process2), axis=1)
        
        X_train_process = np.concatenate((X_train_process, np.ones((X_train_process.shape[0], 1))), axis=1)
        X_test_process = np.concatenate((X_test_process, np.ones((X_test_process.shape[0], 1))), axis=1)
        
    if kernel in ["k_gram", 'k_gram_concat', 'k_gram_concat_several']:
        # Computing the Gram-Matrix
        K = X_train_process.dot(X_train_process.T)
        w = solve_svm(K, Y_train, lamb=lamb, kktreg=kktreg)
        K_test = np.dot(X_test_process, np.transpose(X_train_process))
    
    if kernel=="k_gram_gaussian":
        # Computing the Gram-Matrix
        K = np.array([LA.norm(X_train_process - y, axis=1) for y in X_train_process])
        K = np.exp(-K/gamma)
        # K = X_train_process.dot(X_train_process.T)
        w = solve_svm(K, Y_train, lamb=lamb)
        K_test = np.array([LA.norm(X_train_process - y, axis=1) for y in X_test_process])
        print(K_test.shape)
        K_test = np.exp(-K_test/gamma)
    
    if kernel=='k_substring':
        # Computing the Gram-Matrix
        K = np.zeros((N_train, N_train))
        X_train_process=[]
        X_test_process=[]
        for i in tqdm(range(N_train)):
            X_train_process.append(transform_sequence(X_train[i]))
        for i in tqdm(range(N_test)):
            X_test_process.append(transform_sequence(X_test[i]) )
        
        for i in tqdm(range(len(X_train))):
            for j in range(i, len(X_train)):
                K[i][j] = K[j][i] = compute_gap_kernel(X_train_process[i][0], X_train_process[j][0], k) + \
                    compute_gap_kernel(X_train_process[i][1], X_train_process[j][1], k) + \
                    compute_gap_kernel(X_train_process[i][2], X_train_process[j][2], k)
        w = solve_svm(K, Y_train, lamb=lamb, kktreg=kktreg)
        K_test = np.zeros((N_test, N_train))
        for i in tqdm(range(len(X_test)), desc="Compution testing Kernel"):
            for j in range(len(X_train)):
                K_test[i][j] = compute_gap_kernel(X_test[i], X_train[j], k)
    
    if kernel=='local_alignement':
        K = np.zeros((N_train, N_train))
        X_train_process=[]
        X_test_process=[]
        for i in tqdm(range(N_train)):
            X_train_process.append(transform_sequence(X_train[i]))
        for i in tqdm(range(N_test)):
            X_test_process.append(transform_sequence(X_test[i]) )
        
        for i in tqdm(range(len(X_train))):
            for j in range(i, len(X_train)):
                K[i][j] = K[j][i] = LA_kernel(X_train_process[i][0], X_train_process[j][0]) + \
                    LA_kernel(X_train_process[i][1], X_train_process[j][1]) + \
                    LA_kernel(X_train_process[i][2], X_train_process[j][2])
        w = solve_svm(K, Y_train, lamb=lamb, kktreg=kktreg)
        K_test = np.zeros((N_test, N_train))
        for i in tqdm(range(len(X_test)), desc="Compution testing Kernel"):
            for j in range(len(X_train)):
                K_test[i][j] = LA_kernel(X_test[i], X_train[j])
    
    if kernel=='mismatch':
        # compute all possible k_gram from ATGC
        gramList, gramDict = k_gramGen(k)    
        if (EDITDIST is None):
            # compute dynamically the mismatch distances for all possible k_gram of the alphabet
            editDist = dynMismatchDist(k, gramList = gramList, gramDict = gramDict)
        else:
            editDist = EDITDIST
        # compute the features 
        X_train_preprocess = mismatchFeatures(X_train, k = k, m = m, 
                                             gramList = gramList, gramDict = gramDict, mismatchDist = editDist)
        X_test_preprocess = mismatchFeatures(X_test, k = k, m = m, 
                                             gramList = gramList, gramDict = gramDict, mismatchDist = editDist)
        
        # Adding 1 for the sake of the bias
        X_train_preprocess = np.concatenate((X_train_preprocess, np.ones((X_train_preprocess.shape[0], 1))), axis=1)
        X_test_preprocess = np.concatenate((X_test_preprocess, np.ones((X_test_preprocess.shape[0], 1))), axis=1)
    
        # compute gram matrices from features and solve svm
        K = X_train_preprocess.dot(X_train_preprocess.T)
        K_test = X_test_preprocess.dot(X_train_preprocess.T)
        w = solve_svm(K, Y_train, lamb=lamb, kktreg = kktreg)

    
    if kernel=='diMismatch':
        # compute all possible k_gram from ATGC
        gramList, gramDict = k_gramGen(k)    
        if (EDITDIST is None):
            # compute dynamically the diMismatch distances for all possible k_gram of the alphabet
            editDist = dynDiMismatchDist(k, gramList = gramList, gramDict = gramDict)
        else:
            editDist = EDITDIST
        # compute the features 
        X_train_preprocess = diMismatchFeatures(X_train, k = k, m = m, 
                                             gramList = gramList, gramDict = gramDict, diMismatchDist = editDist)
        X_test_preprocess = diMismatchFeatures(X_test, k = k, m = m, 
                                             gramList = gramList, gramDict = gramDict, diMismatchDist = editDist)
        # Adding 1 for the sake of the bias
        X_train_preprocess = np.concatenate((X_train_preprocess, np.ones((X_train_preprocess.shape[0], 1))), axis=1)
        X_test_preprocess = np.concatenate((X_test_preprocess, np.ones((X_test_preprocess.shape[0], 1))), axis=1)
    
        # compute gram matrices from features and solve svm
        K = X_train_preprocess.dot(X_train_preprocess.T)
        K_test = X_test_preprocess.dot(X_train_preprocess.T)
        w = solve_svm(K, Y_train, lamb=lamb, kktreg=kktreg)

        
    n = K.shape[0]
    Y_predicted = np.dot(K_test, w[:n]) > 0.
    result = ((Y_test+1.)/ 2. == np.transpose(Y_predicted))
    Y_predicted_train = np.dot(K, w[:n]) > 0.
    result_train = ((Y_train+1)/ 2 == np.transpose(Y_predicted_train))
    if np.alltrue(Y_predicted):
        print("Toute les valeurs sont TRUE")
    if np.alltrue(Y_predicted==False):
        print("Toute les valeurs sont FALSE")
    return np.mean(result), np.mean(result_train)
             


In [6]:
def edit_distance(str1, str2):
    """
    Compute the Edit Distance between str1 and str2.
    Return the number of insertion/deletion/substitution
    """
    n = len(str1)
    m = len(str2)
    table_str = np.zeros((n+1,m+1))

    table_str[0, :]=np.arange(m+1)
    table_str[:, 0] = np.arange(n+1)

    for i in range(1, n+1):
        for j in range(1, m+1):
            if(str1[i-1] == str2[j-1]):
                table_str[i, j] = min(table_str[i-1, j] + 1, table_str[i, j-1] + 1, table_str[i-1, j-1]) 
            else:
                table_str[i, j] = min(table_str[i-1, j] + 1, table_str[i, j-1] + 1, table_str[i-1, j-1]+2) 
                
    return table_str

X1 = 'ACCCTG'
X2 = 'AGGTC'
print(edit_distance(X1, X2))

[[ 0.  1.  2.  3.  4.  5.]
 [ 1.  0.  1.  2.  3.  4.]
 [ 2.  1.  2.  3.  4.  3.]
 [ 3.  2.  3.  4.  5.  4.]
 [ 4.  3.  4.  5.  6.  5.]
 [ 5.  4.  5.  6.  5.  6.]
 [ 6.  5.  4.  5.  6.  7.]]


In [7]:
# With Cross validation

X_train0, Y_train0, X_test0, Y_test0 = preprocessing(X_raw0, Y0, percent=0.9)
X_train1, Y_train1, X_test1, Y_test1 = preprocessing(X_raw1, Y1, percent=0.9)
X_train2, Y_train2, X_test2, Y_test2 = preprocessing(X_raw2, Y2, percent=0.9)
print('train shape', X_train0.shape)
print('test shape', X_test0.shape)

print('train shape', X_train1.shape)
print('test shape', X_test1.shape)

print('train shape', X_train2.shape)
print('test shape', X_test2.shape)


train shape (1800,)
test shape (200,)
train shape (1800,)
test shape (200,)
train shape (1800,)
test shape (200,)


In [8]:
k = 6
#for mismatch kernel
#EDITDIST = dynMismatchDist(k)
#for diMismatch kernel
EDITDIST = dynDiMismatchDist(k)

100%|██████████| 4096/4096 [00:15<00:00, 272.94it/s]


In [9]:
acc_test0, acc_train0 = solve_svm_kernel(X_train0,
                                       X_test0,
                                       Y_train0,
                                       Y_test0,
                                       kernel='diMismatch',
                                       k = 6, m = 3,
                                       lamb=0.1)

print('accuracy for train : {}'.format(acc_train0))
print('accuracy for test : {}'.format(acc_test0))

Building Mismatch table for substrings...


100%|██████████| 1800/1800 [00:06<00:00, 288.68it/s]


Building Mismatch table for substrings...


100%|██████████| 200/200 [00:00<00:00, 277.93it/s]


     pcost       dcost       gap    pres   dres
 0:  1.8659e+00  2.9627e+00  4e+03  1e+00  7e+06
 1:  2.7141e+00 -8.8770e+01  9e+01  4e-02  2e+05
 2:  1.6298e+00 -1.4041e+01  2e+01  6e-03  3e+04
 3:  1.2921e+00 -1.6930e+00  3e+00  8e-04  4e+03
 4:  8.6794e-01  1.0798e-01  8e-01  1e-04  6e+02
 5:  4.4350e-01  3.4861e-01  9e-02  5e-06  2e+01
 6:  4.0381e-01  3.6862e-01  4e-02  2e-06  8e+00
 7:  3.8748e-01  3.7700e-01  1e-02  4e-07  2e+00
 8:  3.8163e-01  3.8005e-01  2e-03  5e-08  2e-01
 9:  3.8072e-01  3.8054e-01  2e-04  4e-09  2e-02
10:  3.8061e-01  3.8060e-01  1e-05  7e-10  1e-03
11:  3.8061e-01  3.8061e-01  3e-07  7e-10  2e-05
12:  3.8061e-01  3.8061e-01  8e-09  7e-10  3e-07
13:  3.8061e-01  3.8061e-01  4e-10  7e-10  4e-09
Optimal solution found.
accuracy for train : 0.9022222222222223
accuracy for test : 0.75


In [13]:
acc_test1, acc_train1 = solve_svm_kernel(X_train1, 
                                       X_test1, 
                                       Y_train1, 
                                       Y_test1, 
                                       kernel='diMismatch', 
                                       k=6, m = 1,
                                       lamb=0.001)

print('accuracy for train : {}'.format(acc_train1))
print('accuracy for test : {}'.format(acc_test1))

Building Mismatch table for substrings...


100%|██████████| 1800/1800 [00:08<00:00, 212.68it/s]


Building Mismatch table for substrings...


100%|██████████| 200/200 [00:00<00:00, 234.48it/s]


     pcost       dcost       gap    pres   dres
 0:  3.0405e-02  1.0312e+00  4e+03  1e+00  8e+05
 1:  1.0199e+00 -3.6552e+01  4e+01  1e-02  8e+03
 2:  9.3993e-01 -1.1764e+00  2e+00  5e-04  3e+02
 3:  5.3033e-01 -3.4275e-01  9e-01  2e-04  9e+01
 4:  2.5629e-01 -1.1327e-01  4e-01  6e-05  3e+01
 5:  9.1784e-02 -2.6501e-02  1e-01  2e-05  9e+00
 6:  3.0950e-02 -1.5249e-03  3e-02  4e-06  2e+00
 7:  7.6933e-03  5.8783e-03  2e-03  2e-09  9e-10
 8:  6.7497e-03  6.3474e-03  4e-04  2e-09  2e-10
 9:  6.5285e-03  6.4781e-03  5e-05  2e-09  2e-11
10:  6.4991e-03  6.4960e-03  3e-06  2e-09  5e-12
11:  6.4973e-03  6.4972e-03  1e-07  2e-09  1e-12
12:  6.4972e-03  6.4972e-03  4e-09  2e-09  3e-13
Optimal solution found.
accuracy for train : 1.0
accuracy for test : 0.835


In [15]:
acc_test2, acc_train2 = solve_svm_kernel(X_train2, 
                                       X_test2, 
                                       Y_train2, 
                                       Y_test2, 
                                       kernel='diMismatch',
                                       k = 6, m = 2,
                                       lamb = 0.1)

print('accuracy for train : {}'.format(acc_train2))
print('accuracy for test : {}'.format(acc_test2))

Building Mismatch table for substrings...


100%|██████████| 1800/1800 [00:08<00:00, 209.63it/s]


Building Mismatch table for substrings...


100%|██████████| 200/200 [00:00<00:00, 263.20it/s]


     pcost       dcost       gap    pres   dres
 0:  1.9536e+00  3.0229e+00  4e+03  1e+00  2e+06
 1:  2.8404e+00 -7.7509e+01  8e+01  3e-02  5e+04
 2:  1.7533e+00 -1.5309e+01  2e+01  6e-03  1e+04
 3:  1.3953e+00 -2.2270e+00  4e+00  1e-03  2e+03
 4:  1.0752e+00  1.4745e-01  9e-01  1e-04  2e+02
 5:  5.2768e-01  4.2818e-01  1e-01  1e-05  2e+01
 6:  4.8356e-01  4.6044e-01  2e-02  2e-06  3e+00
 7:  4.7250e-01  4.6769e-01  5e-03  2e-07  3e-01
 8:  4.6984e-01  4.6926e-01  6e-04  2e-08  3e-02
 9:  4.6950e-01  4.6947e-01  3e-05  9e-10  1e-03
10:  4.6948e-01  4.6948e-01  2e-06  7e-10  6e-05
11:  4.6948e-01  4.6948e-01  6e-08  8e-10  1e-06
12:  4.6948e-01  4.6948e-01  2e-09  8e-10  2e-08
Optimal solution found.
accuracy for train : 0.8766666666666667
accuracy for test : 0.59


## Best parameters found for k_gram kernel
#### k = 3
* Set0: lambda=1e-3, gamma = 5 (gaussian), score = 0.68
* Set1: lambda=0.01, score = 
* Set2: lambda= 1e-5, score = 

#### k = 6

* Set0: lamda=0.04 , score = 0.79 (on test)
* Set1: lambda=0.01, score= 0.85 (on test)
* Set2: lambda= 1e-1, score = 0.6875 (on test)

#### k=3 et k=6 (concatenate)

* Set0: lamda=0.04, score = 0.7425 (on test)
* Set1: lambda=0.05, score = 0.89 (on test)

#### k = 3,4,5,6 (concatenate)

* Set1: lambda=0.05, score = 

### mismatch kernel

* Set0: lambda = 1, k = 6, m = 2, score = 0.74
* Set1: lambda = 0.04, k = 6, m = 4, score = 0.85 
* Set2: lambda = 0.04, k = 6 , m = 4, score = 0.645

### diMismatch kernel

* Set0: lambda = 0.1, k = 6, m = 3, score = 0.75
* Set1: lambda = 0.001, k = 6, m = 1, score = 0.83 
* Set2: lambda = 0.1, k = 6 , m = 2, score = 0.59

In [32]:
def solve_svm_test(X_train, X_test, Y_train, kernel='k_gram_gaussian', k=3, k2=6, lamb=0.1, gamma=0.1, kktreg=1e-9):
    """
    kernel in [k_gram, k_gram_gaussian, k_gram_concat, k_gram_concat_several]
    """
    Y_train_process = (Y_train-0.5) * 2
    if kernel in ['k_gram', 'k_gram_gaussian']:
        if k == 3:
            X_train_process = np.array([count_kuplet_3(x) for x in X_train])
            X_test_process = np.array([count_kuplet_3(x) for x in X_test])
        else:
            X_train_process = np.array([count_kuplet_k(x,k=k) for x in X_train])
            X_test_process = np.array([count_kuplet_k(x,k=k) for x in X_test])
        
        # Adding 1 for the sake of the bias
        X_train_process = np.concatenate((X_train_process, np.ones((X_train_process.shape[0], 1))), axis=1)
        X_test_process = np.concatenate((X_test_process, np.ones((X_test_process.shape[0], 1))), axis=1)
    
      
    if kernel=='k_gram_concat':
        print("building kernel...")
        X_train_process1 = np.array([count_kuplet_3(x) for x in X_train])
        X_train_process2 = np.array([count_kuplet_k(x, k=k2) for x in X_train])
        X_train_process = np.concatenate((X_train_process1, X_train_process2), axis=1)
        X_test_process1 = np.array([count_kuplet_3(x) for x in X_test])
        X_test_process2 = np.array([count_kuplet_k(x, k=k2) for x in X_test])
        X_test_process = np.concatenate((X_test_process1, X_test_process2), axis=1)
        
        X_train_process = np.concatenate((X_train_process, np.ones((X_train_process.shape[0], 1))), axis=1)
        X_test_process = np.concatenate((X_test_process, np.ones((X_test_process.shape[0], 1))), axis=1)
    
    if kernel=='k_gram_concat_several':
        print("building kernel...")
        X_train_process = np.array([count_kuplet_3(x) for x in X_train])
        X_test_process = np.array([count_kuplet_3(x) for x in X_test])
        
        for k_tmp in range(4, k2):
            X_train_process2 = np.array([count_kuplet_k(x, k=k_tmp) for x in X_train])
            X_train_process = np.concatenate((X_train_process, X_train_process2), axis=1)
            X_test_process2 = np.array([count_kuplet_k(x, k=k_tmp) for x in X_test])
            X_test_process = np.concatenate((X_test_process, X_test_process2), axis=1)
        
        X_train_process = np.concatenate((X_train_process, np.ones((X_train_process.shape[0], 1))), axis=1)
        X_test_process = np.concatenate((X_test_process, np.ones((X_test_process.shape[0], 1))), axis=1)
    
        
    if kernel in ["k_gram", 'k_gram_concat', 'k_gram_concat_several']:
        # Computing the Gram-Matrix
        K = X_train_process.dot(X_train_process.T)
        w = solve_svm(K, Y_train_process, lamb=lamb, kktreg=kktreg)
        K_test = np.dot(X_test_process, np.transpose(X_train_process))
    
    if kernel=="k_gram_gaussian":
        # Computing the Gram-Matrix
        K = np.array([LA.norm(X_train_process - y, axis=1) for y in X_train_process])
        K = np.exp(-K/gamma)
        # K = X_train_process.dot(X_train_process.T)
        w = solve_svm(K, Y_train_process, lamb=lamb)
        K_test = np.array([LA.norm(X_train_process - y, axis=1) for y in X_test_process])
        K_test = np.exp(-K_test/gamma)
    
    n = K.shape[0]
    Y_predicted = np.dot(K_test, w[:n]) > 0.
    Y_predicted = (Y_predicted + 0.)
    Y_predicted_train = np.dot(K, w[:n]) > 0.
    result_train = (Y_train == np.transpose(Y_predicted_train))
    print('accuracy on train : {}'.format(np.mean(result_train)))
    # result = ((Y_test+1)/ 2 == np.transpose(Y_predicted))
    
    return np.transpose(Y_predicted) # , np.transpose(Y_predicted_train)


In [33]:
print('train shape : {}'.format(X_raw0.shape))
print('train shape 2 : {}'.format(Y0.shape))
print('test shape : {}'.format(X_valid0.shape))

Y_predict0 = solve_svm_test(X_raw0, 
                           X_valid0, 
                           Y0, 
                           kernel='k_gram_concat_several', 
                           k2=7, lamb=.05, gamma=5.)

train shape : (2000,)
train shape 2 : (2000,)
test shape : (1000,)
building kernel...
     pcost       dcost       gap    pres   dres
 0:  1.7284e+00  2.8068e+00  4e+03  1e+00  2e+05
 1:  2.6017e+00 -8.6387e+01  9e+01  3e-02  4e+03
 2:  1.5970e+00 -1.2673e+01  1e+01  5e-03  6e+02
 3:  1.2882e+00 -1.4013e+00  3e+00  6e-04  9e+01
 4:  8.2477e-01  2.0239e-01  6e-01  8e-05  1e+01
 5:  4.3690e-01  3.5905e-01  8e-02  9e-06  1e+00
 6:  3.9758e-01  3.8193e-01  2e-02  1e-06  2e-01
 7:  3.8934e-01  3.8678e-01  3e-03  2e-07  2e-02
 8:  3.8790e-01  3.8764e-01  3e-04  1e-08  2e-03
 9:  3.8775e-01  3.8773e-01  2e-05  7e-10  9e-05
10:  3.8774e-01  3.8774e-01  4e-07  7e-10  2e-06
11:  3.8774e-01  3.8774e-01  1e-08  7e-10  3e-08
Optimal solution found.
accuracy on train : 0.913


In [52]:
print('train shape : {}'.format(X_raw1.shape))
print('train shape 2 : {}'.format(Y1.shape))
print('test shape : {}'.format(X_valid1.shape))

Y_predict1 = solve_svm_test(X_raw1, 
                           X_valid1, 
                           Y1, 
                           kernel='k_gram_concat_several', 
                           k2=7, lamb=0.05, gamma=0.05)

train shape : (2000,)
train shape 2 : (2000,)
test shape : (1000,)
building kernel...
     pcost       dcost       gap    pres   dres
 0:  9.7938e-01  2.0192e+00  4e+03  1e+00  1e+06
 1:  1.9161e+00 -7.7520e+01  8e+01  3e-02  2e+04
 2:  1.3560e+00 -1.0344e+01  1e+01  4e-03  3e+03
 3:  1.1510e+00 -1.8317e+00  3e+00  7e-04  6e+02
 4:  7.5097e-01 -1.6302e-01  9e-01  2e-04  1e+02
 5:  2.9919e-01  1.7869e-01  1e-01  8e-06  7e+00
 6:  2.3488e-01  2.0584e-01  3e-02  2e-06  1e+00
 7:  2.2066e-01  2.1312e-01  8e-03  3e-07  3e-01
 8:  2.1640e-01  2.1540e-01  1e-03  1e-08  1e-02
 9:  2.1581e-01  2.1574e-01  7e-05  9e-10  7e-04
10:  2.1577e-01  2.1576e-01  3e-06  9e-10  2e-05
11:  2.1576e-01  2.1576e-01  8e-08  9e-10  3e-07
12:  2.1576e-01  2.1576e-01  3e-09  9e-10  5e-09
Optimal solution found.
accuracy on train : 0.966


In [51]:
print('train shape : {}'.format(X_raw2.shape))
print('train shape 2 : {}'.format(Y2.shape))
print('test shape : {}'.format(X_valid2.shape))

Y_predict2 = solve_svm_test(X_raw2, 
                           X_valid2, 
                           Y2, 
                           kernel='k_gram_concat_several', 
                           k2=7, lamb=0.03, gamma=0.05)

train shape : (2000,)
train shape 2 : (2000,)
test shape : (1000,)
building kernel...
     pcost       dcost       gap    pres   dres
 0:  1.3960e+00  2.4334e+00  4e+03  1e+00  2e+05
 1:  2.3332e+00 -7.2022e+01  8e+01  3e-02  5e+03
 2:  1.6061e+00 -1.1866e+01  1e+01  4e-03  8e+02
 3:  1.3364e+00 -9.8364e-01  2e+00  5e-04  8e+01
 4:  7.9561e-01  3.3059e-01  5e-01  3e-05  4e+00
 5:  4.7751e-01  4.0469e-01  7e-02  4e-06  7e-01
 6:  4.3765e-01  4.2205e-01  2e-02  7e-07  1e-01
 7:  4.2837e-01  4.2620e-01  2e-03  6e-08  1e-02
 8:  4.2701e-01  4.2684e-01  2e-04  4e-09  7e-04
 9:  4.2690e-01  4.2690e-01  9e-06  6e-10  3e-05
10:  4.2690e-01  4.2690e-01  4e-07  7e-10  8e-07
11:  4.2690e-01  4.2690e-01  1e-08  7e-10  1e-08
Optimal solution found.
accuracy on train : 0.899


In [54]:
test0 = Y_predict0[:][0]
test1 = Y_predict1[:][0]
test2 = Y_predict2[:][0]

bound = np.concatenate((test0,test1,test2), axis=0).reshape((-1)).astype(int)
final = pd.DataFrame(np.arange(3000), columns=['Id'])
final['Bound'] = bound
final.to_csv('resultk_6.csv', index= None)

In [55]:
# Check before submit if the final has good shape
print(final)

,Id,Bound
0,0,0
1,1,1
2,2,0
3,3,0
4,4,1
5,5,1
6,6,1
7,7,0
8,8,1
9,9,0
